In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%capture
import numpy as np
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.models import load_model
tf.compat.v1.disable_eager_execution()


from knn_data_generator_online import KNNDataGenerator

import sys
sys.path.insert(1, '../')
from constants import *
labels_txt = list(LABELS.keys())

In [10]:
model = load_model("checkpoints_dist_online\\0909-triplet-online66-0.0300.h5", compile=False)

In [4]:
train_generator = KNNDataGenerator('D:\\jobs\\2020\\sign_language\\datasets\\classifier\\anns_norm\\train',
                                     batch_size=8, use_augment=False)

[INFO] Loading all label in to memory...
0_Idle 58
Hksl_bear 241
Hksl_bicycle-Jsl_bicycle 386
Hksl_carrot 186
Hksl_chef 187
Hksl_coffee 177
Hksl_dog 188
Hksl_dragonfly 175
Hksl_flip_flops 244
Hksl_fortune_teller 171
Hksl_grey 187
Hksl_hat-Jsl_cap 380
Hksl_hearing_aid-Jsl_hearing_aid 383
Hksl_high_heels 230
Hksl_how_are_you 150
Hksl_lipstick 202
Hksl_milk_tea 139
Hksl_motorcycle-Jsl_motorcycle 299
Hksl_post_office 186
Hksl_purple 184
Hksl_shark 198
Hksl_soft_drink 203
Hksl_summer 176
Hksl_sunglasses 259
Hksl_supermarket 97
Hksl_tiger 231
Hksl_tomato 183
Hksl_watermelon 210
Hksl_welcome 200
Hksl_wine 202
Hksl_winter-Jsl_winter 363
Jsl_bird 268
Jsl_brown 268
Jsl_canteen 210
Jsl_carrot 187
Jsl_cat 247
Jsl_department_store 180
Jsl_dog 203
Jsl_draught_beer 245
Jsl_earring 228
Jsl_geta 216
Jsl_goldfish 223
Jsl_gray 248
Jsl_hello_with_one_hand 208
Jsl_hello_with_two_hands 242
Jsl_high_heels 186
Jsl_japanese_radish 204
Jsl_lemon 261
Jsl_milk_tea 192
Jsl_orange_juice 198
Jsl_programmer 191
Jsl_s

In [11]:
results_pose = []
results_face = []
results_hands = []
results_all = []

labels_idx = []


for i in range(200):
    net_inputs, labels = train_generator.__getitem__(0)
    
    # predict and append embeddings.
    feats = model.predict(net_inputs)
    results_all.append(feats)
    results_pose.append(feats[:,:256])
    results_face.append(feats[:,256:320])
    results_hands.append(feats[:,320:])
    
    
    labels_idx.append(labels)
               
               
# save embeddings.
results_pose = np.vstack(results_pose)
results_face = np.vstack(results_face)
results_hands = np.vstack(results_hands)
results_all = np.vstack(results_all)
np.savetxt("vis\\vecs_pose.tsv", results_pose, delimiter='\t', fmt="%1.5f")
np.savetxt("vis\\vecs_face.tsv", results_face, delimiter='\t', fmt="%1.5f")
np.savetxt("vis\\vecs_hands.tsv", results_hands, delimiter='\t', fmt="%1.5f")
np.savetxt("vis\\vecs_all.tsv", results_all, delimiter='\t', fmt="%1.5f")


               
# save label metadata.
labels_idx = np.vstack(labels_idx).astype("int")


out_m = open('vis\\meta.tsv', 'w', encoding='utf-8')
for idx in labels_idx:
    out_m.write(str(labels_txt[idx[0]]) + "\n")
out_m.close()
